## Analyze the dataset

In [16]:
import numpy as np
import pandas as pd
from pathlib import Path
import networkx as nx
import random

In [7]:
SRC       = "human"
TGT       = "fly"
NETSRC    = f"../../../runs/only_few_landmarks/data/intact/{SRC}.s.tsv"
NETTGT    = f"../../../runs/only_few_landmarks/data/intact/{TGT}.s.tsv"
ALIGNMENT = f"../../../runs/only_few_landmarks/temp/outputs/fly_human_lr_0.001_ep_100_svdr_100_nL_100_dthres_10_ialpha_0.7_wB_0.66/isorank.tsv"
ALIGNMENT = Path(ALIGNMENT)

In [107]:
df    = pd.read_csv(ALIGNMENT, sep = "\t")
dfsrc = pd.read_csv(NETSRC, sep = "\t", header=None)
dftgt = pd.read_csv(NETTGT, sep = "\t", header=None)
Gsrc  = nx.from_pandas_edgelist(dfsrc, 0, 1)
Gtgt  = nx.from_pandas_edgelist(dftgt, 0, 1)

In [108]:
srcmusthave = df[SRC].values
tgtmusthave = df[TGT].values

In [109]:
srcnodes = set(random.sample(list(Gsrc.nodes), 4000) + srcmusthave.tolist())
tgtnodes = set(random.sample(list(Gtgt.nodes), 4000) + tgtmusthave.tolist())

In [110]:
Gsubsrc = Gsrc.subgraph(srcnodes)
Gsubtgt = Gtgt.subgraph(tgtnodes)

## Remove high degree nodes

In [112]:
Gsubsrc = Gsubsrc.subgraph([x for x, i in list(Gsubsrc.degree()) if i < 25])
Gsubtgt = Gsubtgt.subgraph([x for x, i in list(Gsubtgt.degree()) if i < 25])

In [113]:
srccc   = max(nx.connected_components(Gsubsrc), key = len)
tgtcc   = max(nx.connected_components(Gsubtgt), key = len)
Gsubsrc = Gsubsrc.subgraph(srccc)
Gsubtgt = Gsubtgt.subgraph(tgtcc)
srcmusthave = set(srccc).intersection(srcmusthave)
tgtmusthave = set(tgtcc).intersection(tgtmusthave)

## Constructing a super-matrix that combines both source and target

In [166]:
smap        = {k: i for i, k in enumerate(Gsubsrc.nodes())}
tmap        = {k: i for i, k in enumerate(Gsubtgt.nodes())}
srcmat      = nx.adjacency_matrix(Gsubsrc).toarray()
tgtmat      = nx.adjacency_matrix(Gsubtgt).toarray()
tgt_src_mat = np.zeros((tgtmat.shape[0], srcmat.shape[0]))
SC          = 1
for sc, tg in df[[SRC, TGT]].values:
    try:
        idxsc = smap[sc]
        idxtg = tmap[tg]
    except Exception as e:
        continue
    tgt_src_mat[idxtg, idxsc] = SC 

In [167]:
combinedmat = np.zeros((srcmat.shape[0]+tgtmat.shape[0], srcmat.shape[0]+tgtmat.shape[0]))
combinedmat[:tgtmat.shape[0], :tgtmat.shape[0] ] = tgtmat
combinedmat[tgtmat.shape[0]:,  tgtmat.shape[0]:] = srcmat
combinedmat[:tgtmat.shape[0],  tgtmat.shape[0]:] = tgt_src_mat
combinedmat[tgtmat.shape[0]:, :tgtmat.shape[0] ] = tgt_src_mat.T

## Finally generating the DSD matrix

In [168]:
import glidetools.algorithm.dsd as dsd
CDSD = dsd.compute_dsd_embedding(combinedmat, is_normalized=True)

In [169]:
from scipy.spatial.distance import pdist, squareform

In [170]:
DIST = squareform(pdist(CDSD))

## Extract the upper right matrix representing the relationship between two species, and select the five top locations

In [171]:
UPPER_RIGHT = DIST[:tgtmat.shape[0], tgtmat.shape[0]:]

In [172]:
UPPER_RIGHT[:, [1743,  866, 1919,  685, 1614]]

array([[4.33847883, 4.39528993, 3.9170962 , 4.30292799, 3.41438467],
       [3.53955654, 3.60620126, 3.03504249, 3.49745051, 2.3981487 ],
       [3.52987218, 3.59490361, 3.01062793, 3.48556454, 2.39133412],
       ...,
       [3.87752393, 3.94827952, 3.37220896, 3.83665492, 2.74599076],
       [3.91728216, 3.99889533, 3.46470226, 3.89686416, 2.91132905],
       [4.0272935 , 4.09593216, 3.54060506, 3.98761223, 2.94664583]])

In [173]:
CLOSEST

array([[1743,  866, 1614, ..., 1209, 1840, 2088],
       [1743,  866, 1919, ..., 1205, 1136,  458],
       [1743,  866, 1919, ..., 1209, 1205, 1136],
       ...,
       [1743,  866, 1919, ...,  362,  905, 1764],
       [1743,  866, 1919, ..., 1205, 1262,  129],
       [1743,  866, 1919, ...,  441,  362,  905]])

In [174]:
CLOSEST = np.argsort(UPPER_RIGHT, axis = 1)[:, :250]
rtmap = {v: k for k, v in tmap.items()}
rsmap = {v: k for k, v in smap.items()}
clmat   = []
columns = ["target"] + [f"src-{idx}" for idx in range(250)]
for i, entries in enumerate(CLOSEST):
    clmat.append([rtmap[i]] + [rsmap[j] for j in entries])
pd.DataFrame(clmat, columns = columns).set_index("target").to_csv(f"{SRC}_{TGT}_{SC}.tsv", sep = "\t")